# AGN Deep Learning Classification for EDFN

The purpose of this notebook is explore classifying AGN in the EDFN catalog using the X-ray data in the matched EDFF catalogs. Ensure that the selected kernel is running the agnml environment which runs python 3.10 for tensorflow

In [152]:
import sys
sys.path.append("./utils")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List
from numpy import ndarray


import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D, Concatenate, Input
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sed_utils import SEDUtils
from config_loader import ConfigLoader
from data_loader import DataLoader

In [153]:
config = ConfigLoader("/Users/admin/Documents/GitHub/Matched-Catalog-Analysis/src/configs/config.yaml")
data = DataLoader(verbose = 1)
sed = SEDUtils
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Building Datasets

In [154]:
edff_sed_data = pd.read_csv(config.EDFF_SED_DATA)
edff_sed_data.head()

,edffID,z,F_x,F_u,F_g,F_r,F_i,F_z,F_ch1,F_ch2,L_x,L_u,L_g,L_r,L_i,L_z,L_ch1,L_ch2
0,458822,1.628,1.100000e-14,1.348051e-14,2.407560e-14,1.446590e-14,1.659366e-14,1.780411e-14,1.688532e-14,1.703644e-14,1.947539e+44,2.386710e+44,4.262561e+44,2.561173e+44,2.937890e+44,3.152201e+44,2.989530e+44,3.016285e+44
1,432521,2.713,2.700000e-15,2.967598e-14,2.523562e-14,3.314206e-14,2.295913e-14,1.844894e-14,8.835780e-15,7.497009e-15,1.669942e+44,1.835450e+45,1.560815e+45,2.049826e+45,1.420015e+45,1.141061e+45,5.464902e+44,4.636876e+44
2,435358,0.731,8.700000e-16,6.125355e-15,9.202260e-15,1.481001e-14,2.503499e-14,2.919349e-14,2.085041e-14,1.174079e-14,2.120544e+42,1.492999e+43,2.242965e+43,3.609802e+43,6.102047e+43,7.115643e+43,5.082093e+43,2.861708e+43
3,455154,1.888,8.700000e-15,1.769347e-13,6.805683e-14,8.287486e-14,8.323912e-14,5.787950e-14,2.950656e-14,3.081602e-14,2.219682e+44,4.514238e+45,1.736374e+45,2.114435e+45,2.123728e+45,1.476713e+45,7.528181e+44,7.862271e+44
4,454343,1.404,2.900000e-15,4.451685e-15,4.544801e-15,7.163000e-15,1.220730e-14,1.668676e-14,3.102924e-14,3.081073e-14,3.559166e+43,5.463548e+43,5.577828e+43,8.791140e+43,1.498201e+44,2.047964e+44,3.808214e+44,3.781397e+44


## Model Definition

In [155]:
class TransformerModel:
    def __init__(self, input_shape, num_layers, d_model, num_heads, dff, maximum_position_encoding, dropout_rate=0.1, learning_rate=0.001):
        self.sequence_scaler = MinMaxScaler()
        self.scalar_scaler = MinMaxScaler()
        self.target_scaler = MinMaxScaler()  # Initialize target scaler

        self.model = self.build_model(input_shape, num_layers, d_model, num_heads, dff, maximum_position_encoding, dropout_rate)
        self.model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    def build_model(self, input_shape, num_layers, d_model, num_heads, dff, maximum_position_encoding, dropout_rate):
        sequence_input = Input(shape=input_shape)
        scalar_input = Input(shape=(1,))

        encoder = Encoder(num_layers, d_model, num_heads, dff, maximum_position_encoding, dropout_rate)
        encoder_output = encoder(sequence_input, training=False, mask=None)
        
        pooled_output = GlobalAveragePooling1D()(encoder_output)
        scalar_dense = Dense(d_model, activation='relu')(scalar_input)
        
        concatenated = Concatenate()([pooled_output, scalar_dense])
        output = Dense(1, activation='linear')(concatenated)
        
        model = Model(inputs=[sequence_input, scalar_input], outputs=output)
        return model

    def log_transform(self, values):
        return np.log(values) / np.log(14)  # Convert values to numpy array for log10 transformation

    def inverse_log_transform(self, values):
        return np.power(14, values)

    def fit(self, sequences, scalars, targets, test_size=0.2, epochs=50, batch_size=32):
        # Log transform the sequences, scalar values, and targets
        log_sequences = [self.log_transform(seq) for seq in sequences]
        log_scalars = self.log_transform(np.array(scalars).reshape(-1, 1))
        log_targets = self.log_transform(np.array(targets).reshape(-1, 1))
        
        # Normalize the log-transformed values
        norm_sequences = [self.sequence_scaler.fit_transform(seq) for seq in log_sequences]
        norm_scalars = self.scalar_scaler.fit_transform(log_scalars)
        norm_targets = self.target_scaler.fit_transform(log_targets)
        
        # Pad the sequences
        padded_sequences = pad_sequences(norm_sequences, padding='post', dtype='float32')
        padded_sequences = tf.convert_to_tensor(padded_sequences, dtype=tf.float32)
        norm_scalars = tf.convert_to_tensor(norm_scalars, dtype=tf.float32)
        norm_scalars = tf.reshape(norm_scalars, (-1, 1))

        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(
            list(zip(padded_sequences, norm_scalars)), norm_targets, test_size=test_size
        )
        
        X_train_sequences, X_train_scalars = zip(*X_train)
        X_test_sequences, X_test_scalars = zip(*X_test)
        
        X_train_sequences = tf.convert_to_tensor(X_train_sequences, dtype=tf.float32)
        X_train_scalars = tf.convert_to_tensor(X_train_scalars, dtype=tf.float32)
        X_test_sequences = tf.convert_to_tensor(X_test_sequences, dtype=tf.float32)
        X_test_scalars = tf.convert_to_tensor(X_test_scalars, dtype=tf.float32)
        
        y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
        y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

        # Print the shapes of the training and testing data
        print("X_train_sequences shape:", X_train_sequences.shape)
        print("X_train_scalars shape:", X_train_scalars.shape)
        print("y_train shape:", y_train.shape)
        print("X_test_sequences shape:", X_test_sequences.shape)
        print("X_test_scalars shape:", X_test_scalars.shape)
        print("y_test shape:", y_test.shape)

        # Train the model
        self.model.fit([X_train_sequences, X_train_scalars], y_train, epochs=epochs, batch_size=batch_size, validation_data=([X_test_sequences, X_test_scalars], y_test))
    
    def predict(self, sequence, scalar):
        log_sequence = self.log_transform(sequence)
        log_scalar = self.log_transform(np.array([scalar]).reshape(-1, 1))
        
        norm_sequence = self.sequence_scaler.transform(log_sequence)
        norm_scalar = self.scalar_scaler.transform(log_scalar)
        
        padded_sequence = pad_sequences([norm_sequence], padding='post', dtype='float32')
        padded_sequence = tf.convert_to_tensor(padded_sequence, dtype=tf.float32)
        norm_scalar = tf.convert_to_tensor(norm_scalar, dtype=tf.float32)
        norm_scalar = tf.reshape(norm_scalar, (-1, 1))

        log_prediction = self.model.predict([padded_sequence, norm_scalar])
        prediction = self.inverse_log_transform(self.target_scaler.inverse_transform(log_prediction))[0][0]
        return prediction

# Helper Classes for Transformer Model
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.split_heads(self.wq(q), batch_size)
        k = self.split_heads(self.wk(k), batch_size)
        v = self.split_heads(self.wv(v), batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)
        return output, attention_weights

def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2

class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding, dropout_rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Dense(d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, dropout_rate) for _ in range(self.num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x += self.pos_encoding[:seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x

def positional_encoding(position, d_model):
    angle_rads = np.arange(position)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / d_model)
    sines = np.sin(angle_rads[:, 0::2])
    cosines = np.cos(angle_rads[:, 1::2])
    pos_encoding = np.concatenate([sines, cosines], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

def scaled_dot_product_attention(query, key, value, mask):
    matmul_qk = tf.matmul(query, key, transpose_b=True)
    dk = tf.cast(tf.shape(key)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, value)
    return output, attention_weights

## Data Preprocessing

In [156]:
#in Angstroms
LAMBDA_MID_X = (6.19 + 24.80 ) / 2 
LAMBDA_MID_U = (3147.51	+ 4018.39) / 2
LAMBDA_MID_G = (3942.85 + 5547.08) / 2
LAMBDA_MID_R2 = (5021.21 + 7216.76) / 2
LAMBDA_MID_I = (6440.47 + 8643.90) / 2
LAMBDA_MID_Z = (8564.13 + 9490.57) / 2
LAMBDA_MID_CH1 = (31296.24 + 39614.36) / 2
LAMBDA_MID_CH2 = (39173.28 + 50560.57) / 2

wavelength_dict = {
    'LAMBDA_MID_X': LAMBDA_MID_X,
    'LAMBDA_MID_U': LAMBDA_MID_U,
    'LAMBDA_MID_G': LAMBDA_MID_G,
    'LAMBDA_MID_R2': LAMBDA_MID_R2,
    'LAMBDA_MID_I': LAMBDA_MID_I,
    'LAMBDA_MID_Z': LAMBDA_MID_Z,
    'LAMBDA_MID_CH1': LAMBDA_MID_CH1,
    'LAMBDA_MID_CH2': LAMBDA_MID_CH2
}


training_data_dict: dict = {}

target_flux_list = []
input_wavelength_list = []
flux_sequence_list = []
for _, row in edff_sed_data.iterrows():
    wavelengths_rest = np.array([wavelength_dict[w] / (1e4 * (1 + row['z'])) for w in wavelength_dict])
    fluxes = row[['F_x','F_u', 'F_g', 'F_r', 'F_i', 'F_z', 'F_ch1', 'F_ch2']].values
    
    flux_sequence: List[ndarray] = []
    for i, _ in enumerate(wavelengths_rest[1:]): #exclude x-ray data
        flux_wavelength_pair = np.array([fluxes[i], wavelengths_rest[i]])
        flux_sequence.append(flux_wavelength_pair)
    
    target_flux_list.append(fluxes[0])
    input_wavelength_list.append(wavelengths_rest[0])
    flux_sequence_list.append(flux_sequence)


training_data_dict['target_flux_list'] = target_flux_list
training_data_dict['input_wavelength_list'] = input_wavelength_list
training_data_dict['flux_sequence_list'] = flux_sequence_list
        
print(training_data_dict['flux_sequence_list'])

[[array([1.10000000e-14, 5.89611872e-04]), array([1.34805053e-14, 1.36337519e-01]), array([2.40756029e-14, 1.80554224e-01]), array([1.44659017e-14, 2.32838090e-01]), array([1.65936567e-14, 2.86993341e-01]), array([1.78041147e-14, 3.43506469e-01]), array([1.68853241e-14, 1.34913623e+00])], [array([2.70000000e-15, 4.17317533e-04]), array([2.96759778e-14, 9.64974414e-02]), array([2.52356164e-14, 1.27793294e-01]), array([3.31420622e-14, 1.64798950e-01]), array([2.29591336e-14, 2.03129141e-01]), array([1.84489355e-14, 2.43128198e-01]), array([8.83578024e-15, 9.54896310e-01])], [array([8.70000000e-16, 8.95147314e-04]), array([6.12535541e-15, 2.06987291e-01]), array([9.20225970e-15, 2.74116984e-01]), array([1.48100074e-14, 3.53494223e-01]), array([2.50349899e-14, 4.35712594e-01]), array([2.91934912e-14, 5.21510687e-01]), array([2.08504056e-14, 2.04825534e+00])], [array([8.70000000e-15, 5.36530471e-04]), array([1.76934698e-13, 1.24063366e-01]), array([6.80568319e-14, 1.64299342e-01]), array([8

## Model Training

In [160]:
flux_sequence_list = training_data_dict['flux_sequence_list']
input_wavelength_list = training_data_dict['input_wavelength_list']
target_flux_list = training_data_dict['target_flux_list']

# Split data into training and validation sets
flux_train, flux_val, wavelength_train, wavelength_val, target_train, target_val = train_test_split(
    flux_sequence_list, input_wavelength_list, target_flux_list, test_size=0.1, random_state=42
)

# Print lengths to confirm splitting
print("Training data length:", len(flux_train))
print("Validation data length:", len(flux_val))

# Instantiate the TransformerModel
model = TransformerModel(input_shape=(None, 2), num_layers=100, d_model=128, num_heads=64, dff=512, maximum_position_encoding=10000, dropout_rate=0.2)
model.model.summary()

# Train the model
model.fit(flux_train, wavelength_train, target_train, epochs=10, batch_size=4)
predictions = [model.predict(seq, scalar) for seq, scalar in zip(flux_val, wavelength_val)]

results_df = pd.DataFrame({
    'Prediction': predictions,
    'Actual': target_val
})

Training data length: 461
Validation data length: 52
Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_43 (InputLayer)          [(None, None, 2)]    0           []                               
                                                                                                  
 encoder_21 (Encoder)           (None, None, 128)    19827584    ['input_43[0][0]']               
                                                                                                  
 input_44 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 global_average_pooling1d_21 (G  (None, 128)         0           ['encoder_21[0][0]']             
 lobalAveragePooling1D)               

In [158]:
print(results_df)

      Prediction        Actual
0   2.548539e-16  2.900000e-17
1   1.762226e-16  1.503928e-14
2   3.017885e-16  9.700000e-17
3   2.245731e-16  5.200000e-17
4   2.048504e-16  5.509602e-14
5   2.781889e-16  2.800000e-17
6   2.115172e-16  4.700000e-17
7   2.831598e-16  5.921192e-14
8   2.650457e-16  1.100000e-17
9   2.226277e-16  1.000000e-17
10  2.271207e-16  4.700000e-17
11  2.074811e-16  2.700000e-17
12  2.295100e-16  7.900000e-17
13  2.186231e-16  6.100000e-17
14  2.608468e-16  7.200000e-15
15  2.179847e-16  3.100000e-17
16  2.113714e-16  1.300000e-16
17  2.113709e-16  2.200000e-16
18  2.547917e-16  1.300000e-17
19  2.434337e-16  2.300000e-17
20  2.251260e-16  1.400000e-17
21  2.182685e-16  1.400000e-17
22  1.827148e-16  2.600000e-17
23  2.946251e-16  3.200000e-17
24  2.109340e-16  4.100000e-15
25  2.207385e-16  1.900000e-17
26  2.294424e-16  4.600000e-17
27  1.888023e-16  4.200000e-16
28  3.342506e-16  3.000000e-17
29  1.761968e-16  1.500000e-16
30  2.409644e-16  2.200000e-17
31  2.46

In [159]:
edff = data.get_dataframe(config.EDFF_PATH)
edff.head(10)

Filename: /Users/admin/Documents/GitHub/Matched-Catalog-Analysis/src/data_raw/DAWN_EDFF_DR1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    268   1062645R x 130C   [K, D, D, D, D, D, D, D, D, 43A, K, K, D, 20A, E, D, D, D, D, D, D, L, D, D, D, D, D, D, L, D, D, D, D, D, D, L, D, D, D, D, D, D, L, D, D, D, D, D, D, L, D, D, D, D, D, D, L, D, D, D, D, D, D, L, D, D, D, D, D, D, L, D, D, D, D, D, D, L, D, D, D, D, D, D, D, K, D, D, K, K, D, K, D, D, K, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, K, D, D, E, D]   


,ID,ALPHA_J2000,DELTA_J2000,X_MODEL,Y_MODEL,ERRX_MODEL,ERRY_MODEL,ALPHA_DETECTION,DELTA_DETECTION,FARMER_ID,GROUP_ID,N_GROUP,MODEL_FLAG,SOLUTION_MODEL,EBV_MW,CFHT_u_FLUX,CFHT_u_FLUXERR,CFHT_u_MAG,CFHT_u_MAGERR,CFHT_u_CHISQ,CFHT_u_DRIFT,CFHT_u_VALID,HSC_g_FLUX,HSC_g_FLUXERR,HSC_g_MAG,HSC_g_MAGERR,HSC_g_CHISQ,HSC_g_DRIFT,HSC_g_VALID,HSC_r_FLUX,HSC_r_FLUXERR,HSC_r_MAG,HSC_r_MAGERR,HSC_r_CHISQ,HSC_r_DRIFT,HSC_r_VALID,HSC_i_FLUX,HSC_i_FLUXERR,HSC_i_MAG,HSC_i_MAGERR,HSC_i_CHISQ,HSC_i_DRIFT,HSC_i_VALID,HSC_z_FLUX,HSC_z_FLUXERR,HSC_z_MAG,HSC_z_MAGERR,HSC_z_CHISQ,HSC_z_DRIFT,HSC_z_VALID,HSC_NB0816_MAG,HSC_NB0816_MAGERR,HSC_NB0816_FLUX,HSC_NB0816_FLUXERR,HSC_NB0816_CHISQ,HSC_NB0816_DRIFT,HSC_NB0816_VALID,HSC_NB0921_MAG,HSC_NB0921_MAGERR,HSC_NB0921_FLUX,HSC_NB0921_FLUXERR,HSC_NB0921_CHISQ,HSC_NB0921_DRIFT,HSC_NB0921_VALID,IRAC_CH1_FLUX,IRAC_CH1_FLUXERR,IRAC_CH1_MAG,IRAC_CH1_MAGERR,IRAC_CH1_CHISQ,IRAC_CH1_DRIFT,IRAC_CH1_VALID,IRAC_CH2_FLUX,IRAC_CH2_FLUXERR,IRAC_CH2_MAG,IRAC_CH2_MAGERR,IRAC_CH2_CHISQ,IRAC_CH2_DRIFT,IRAC_CH2_VALID,lp_zPDF,lp_zPDF_l68,lp_zPDF_u68,lp_zMinChi2,lp_chi2_best,lp_zp_2,lp_chi2_2,lp_NbFilt,lp_zq,lp_chiq,lp_modq,lp_mods,lp_chis,lp_model,lp_age,lp_dust,lp_Attenuation,lp_MNUV,lp_MR,lp_MJ,lp_mass_med,lp_mass_med_min68,lp_mass_med_max68,lp_mass_best,lp_SFR_med,lp_SFR_med_min68,lp_SFR_med_max68,lp_SFR_best,lp_sSFR_med,lp_sSFR_med_min68,lp_sSFR_med_max68,lp_sSFR_best,ez_z_phot,ez_z_phot_chi2,ez_z_phot_risk,ez_z_min_risk,ez_min_risk,ez_z_raw_chi2,ez_raw_chi2,ez_z_ml,ez_z_ml_chi2,ez_z_ml_risk,ez_z025,ez_z160,ez_z500,ez_z840,ez_z975,ez_nusefilt,ez_lc_min,ez_lc_max,ez_star_min_chi2,ez_star_teff
0,1,52.357832,-28.015619,30860.257400,13805.184308,0.085609,0.078303,52.357813,-28.015630,84_10000,7133,1,0.0,DevGalaxy,0.008067,0.244099,0.038200,25.431085,0.170423,0.855771,0.000137,True,0.241856,0.016933,25.441109,0.076242,0.838295,0.002879,True,0.367419,0.018993,24.987096,0.056295,1.214715,0.001433,True,0.553794,0.019933,24.541628,0.039197,0.881066,0.001825,True,0.572930,0.127948,24.504746,0.243198,1.174499,0.000455,True,24.179534,0.288425,0.773012,0.204735,0.994490,0.000235,True,24.766254,0.289006,0.450296,0.119503,0.993660,0.000333,True,0.248496,0.023791,25.411703,0.104259,0.461124,0.002720,True,0.332080,0.024589,25.096892,0.080635,0.345424,0.002907,True,0.2739,0.2271,0.4957,0.25080,3.119610,0.58,8.40127,7,0.10,14.64870,1,140,28.792400,9,5.000000e+07,0.4,1,-14.39824,-15.87910,-15.92102,7.31936,7.10996,7.51830,6.97556,-1.07858,-1.41111,-0.76005,-0.3699,-8.38682,-8.92388,-7.90127,-7.345,0.263709,7.140742,0.286292,0.246077,0.284072,0.221403,6.836875e+00,0.219051,6.828080,0.290870,0.067486,0.155573,0.263709,0.503743,0.647144,7,3640.981999,45020.337852,9.676304,4800.0
1,2,52.319698,-28.015597,31581.719675,13802.165124,0.024598,0.023457,52.319718,-28.015598,84_10001,7117,1,0.0,ExpGalaxy,0.008453,0.580868,0.023028,24.489807,0.043172,1.006324,0.003965,True,0.641079,0.010144,24.382721,0.017231,1.058447,0.019112,True,0.766861,0.012455,24.188208,0.017687,1.130228,0.007700,True,1.382581,0.012933,23.548273,0.010186,1.330757,0.005565,True,1.625042,0.079703,23.372839,0.053412,1.210985,0.004524,True,23.686655,0.125781,1.217133,0.140581,1.036741,0.000713,True,23.346758,0.052938,1.664550,0.080916,0.959715,0.000403,True,1.374851,0.019157,23.554361,0.015174,0.982006,0.035655,True,1.219494,0.019209,23.684551,0.017153,0.456101,0.032404,True,0.8300,0.7862,0.8732,0.83320,17.081100,NaN,NaN,7,0.00,92.67000,4,225,368.737000,5,6.405000e+08,0.1,1,-18.62216,-19.53033,-19.46875,8.88403,8.81660,8.95326,8.89260,0.21810,-0.01433,0.31627,0.2267,-8.65482,-8.99191,-8.53425,-8.666,0.817779,17.086884,0.091938,0.822119,0.091774,0.803988,1.698096e+01,0.807446,16.973167,0.093952,0.108311,0.767913,0.817779,0.866753,0.913370,7,3640.981999,45020.337852,285.494263,4000.0
2,3,52.440821,-28.015964,29290.181053,13804.580551,0.020469,0.019597,52.440827,-28.015956,84_10002,7128,1,0.0,DevGalaxy,0.008042,0.352241,0.018886,25.032899,0.058387,0.614941,0.003633,True,0.598796,0.008660,2